Code to calculate the amount of flux through redox reactions that use NAD(P) as opposed to other redox co-factors, excluding redox reactions that use O2. We calculate this ratio in a variety of carbon sources, all of which are denoted below and in the supplementary information.

1. We use parsimonious FBA (pFBA), which minimizes the total sum of fluxes while ensuring that maximum biomass production is achieved. This method is intended to encapsulate the fact that enzyme expression is not free and is likely minimized even while cells maximize growth. pFBA has been shown to yield more accurate flux predictions than vanilla FBA. See Lewis, N. E. et al. (2010). Omic data from evolved E. coli are consistent with computed optimal growth from genome-scale models. Mol. Syst. Biol. 6, 390.
2. We are using the complete stoichiometric model iJO1366 which comes with CobraPY. See Orth, J. D., Conrad, T. M., Na, J., Lerman, J. a, Nam, H., Feist, A. M., and Palsson, B. Ø. (2011). A comprehensive genome-scale reconstruction of Escherichia coli metabolism—2011. Mol. Syst. Biol. 7, 1–9.
3. Cytochromes are not included directly in the model. Rather, reactions involving cytochromes are written as donating electrons to O2. Therefore, cytochromes could not be included in this analysis. 

In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import cobra.test
from cobra.flux_analysis import parsimonious

In [2]:
def get_redox_reactions(pair, exclude_metabolites=None):
    """Grabs the redox reactions in which this pair participates.
    
    Args:
        pair: 2 tuple of oxidized and reduced form of a redox cofactor.
            order irrelevant.
        exclude_metabolites: an iterable of metabolites.
            if any of these appear in a reaction, the reaction will be
            excluded from the returned list.
    
    Returns:
        A list of reaction IDs.
    """
    first, second = pair
    first_reactions = first.reactions
    second_reactions = second.reactions
    
    if exclude_metabolites:
        exclude_set = set(exclude_metabolites)
        first_reactions = [r for r in first_reactions
                           if not set(r.metabolites.keys()).intersection(exclude_set)]
        second_reactions = [r for r in second_reactions
                            if not set(r.metabolites.keys()).intersection(exclude_set)]
    
    rids = set([r.id for r in first_reactions])
    rids.intersection_update([r.id for r in second_reactions])
    return rids
    

In [3]:
# Enumerate all the relevant redox pairs and their reactions,
# excluding those that utilize O2 (as described in the text).
model = cobra.test.create_test_model("ecoli")
print 'Model %s' % model.description

# Exclude O2
o2 = model.metabolites.get_by_id('o2_c')
exclude = [o2]

# NAD(P)(H)
nad = model.metabolites.get_by_id('nad_c')
nadh = model.metabolites.get_by_id('nadh_c')
nadp = model.metabolites.get_by_id('nadp_c')
nadph = model.metabolites.get_by_id('nadph_c')

nad_reactions = get_redox_reactions((nad, nadh), exclude)
nadp_reactions = get_redox_reactions((nadp, nadph), exclude)
all_nad_reactions = nad_reactions.union(nadp_reactions)

# Flavin carriers
fad = model.metabolites.get_by_id('fad_c')
fadh = model.metabolites.get_by_id('fadh2_c')

fmn = model.metabolites.get_by_id('fmn_c')
fmnh2 = model.metabolites.get_by_id('fmnh2_c')

riboflavin = model.metabolites.get_by_id('ribflv_c')
riboflavin_red = model.metabolites.get_by_id('rbflvrd_c')

riboflavin_reactions = get_redox_reactions((riboflavin, riboflavin_red), exclude)
fad_reactions = get_redox_reactions((fad, fadh), exclude)
fmn_reactions = get_redox_reactions((fmn, fmnh2), exclude)

# Ferredoxin is annotated as iron 2/3
fd_ox = model.metabolites.get_by_id('fe3_c')
fd_red = model.metabolites.get_by_id('fe2_c')
fd_reactions = get_redox_reactions((fd_ox, fd_red), exclude)

# Quinones
menaquinone_8 = model.metabolites.get_by_id('mqn8_c')
menaquinol_8 = model.metabolites.get_by_id('mql8_c')
ubiquinone_8 = model.metabolites.get_by_id('q8_c')
ubiquinol_8 = model.metabolites.get_by_id('q8h2_c')

mena_reactions = get_redox_reactions((menaquinone_8, menaquinol_8), exclude)
ubi_reactions = get_redox_reactions((ubiquinone_8, ubiquinol_8), exclude)

# Disulfide equivalents glutathione, thioredoxin and glutaredoxin
glth_ox = model.metabolites.get_by_id('gthox_c')
glth_red = model.metabolites.get_by_id('gthrd_c')
thio_ox = model.metabolites.get_by_id('trdox_c')
thio_red = model.metabolites.get_by_id('trdrd_c')
glut_ox = model.metabolites.get_by_id('grxox_c')
glut_red = model.metabolites.get_by_id('grxrd_c')

glth_reactions = get_redox_reactions((glth_ox, glth_red), exclude)
thio_reactions = get_redox_reactions((thio_ox, thio_red), exclude)
glut_reactions = get_redox_reactions((glut_ox, glut_red), exclude)

non_nad_rxn_sets = [fad_reactions, fmn_reactions, riboflavin_reactions,
                    fd_reactions, mena_reactions, ubi_reactions,
                    glth_reactions, thio_reactions, glut_reactions]
non_nad_reactions = set().union(*non_nad_rxn_sets)

n_nad = len(all_nad_reactions)
n_non_nad = len(non_nad_reactions)
total_redox_reactions = n_nad + n_non_nad
print('%d reactions using NAD(P) as redox carrier' % n_nad)
print('%d reactions using something other than NAD(P) as redox carrier' % n_non_nad)
print('%d reactions total' % total_redox_reactions)

pct = 100 * n_nad / float(total_redox_reactions)
print('%.1f%% of redox reactions that don\'t use O2 do use NAD(P)' % pct)

Model iJO1366
205 reactions using NAD(P) as redox carrier
103 reactions using something other than NAD(P) as redox carrier
308 reactions total
66.6% of redox reactions that don't use O2 do use NAD(P)


In [4]:
def solve_w_csource(model, csource_rxn_id, max_uptake_rate=10.0, anaerobic=False,
                    **solver_args):
    """Sets the given reaction ID as the carbon source and solves.
    
    Solves using parsimonious FBA.
    
    Args:
        model: metabolic model.
        csource_rxn_id: ID of the import reaction for the carbon source. 
    
    Returns:
        Optimization solution.
    """
    rxn = model.reactions.get_by_id(csource_rxn_id)
    rxn.lower_bound = -max_uptake_rate
    rxn.upper_bound = max_uptake_rate
    
    oxygen_uptake = model.reactions.get_by_id('EX_o2_e')
    if anaerobic:
        oxygen_uptake.lower_bound = 0
        oxygen_uptake.upper_bound = 0
    else:
        oxygen_uptake.lower_bound = -1000
        oxygen_uptake.upper_bound = 1000
    
    try:
        res = parsimonious.optimize_minimal_flux(model, **solver_args)
    except Exception as e:
        print(e)
        print 'csource = %s, anaerobic = %s' % (csource_rxn_id, anaerobic)
        res = None
        
    # Reset lower bound after solving
    rxn.lower_bound = 0.0
    
    return res
    
def calc_redox_fluxes(solution):
    
    if solution is None:
        return None, None, None
    
    fluxes_dict = solution.x_dict
    nad_fluxes = [abs(fluxes_dict.get(r)) for r in all_nad_reactions]
    total_nad_flux = sum(nad_fluxes)
    
    non_nad_fluxes = [abs(fluxes_dict.get(r)) for r in non_nad_reactions]
    total_non_nad_flux = sum(non_nad_fluxes)
    
    fraction_of_total = total_nad_flux / (float(total_nad_flux + total_non_nad_flux))
    
    return total_nad_flux, total_non_nad_flux, fraction_of_total
    
def print_redox_fluxes(solution):
    """Given a model solution, print the redox reaction fluxes."""
    if solution is None:
        print('Model could not be solved')
        return
    
    total_nad_flux, total_non_nad_flux, fraction_of_total = calc_redox_fluxes(solution)
    print 'Total NAD flux', total_nad_flux
    print 'Total Non-NAD flux', total_non_nad_flux
    print 'Fraction of total %.2f' % fraction_of_total

In [5]:
# Set external glucose to 0 so we can test different carbon sources.
# Glucose is the default carbon source. 
glucose_uptake = model.reactions.get_by_id('EX_glc_e')
glucose_uptake.lower_bound = 0.0

csources = {
  6: {'D-glucose': 'EX_glc_e',
      'D-fructose': 'EX_fru_e',
      'D-glucosamine': 'EX_gam_e',
      'D-gluconate': 'EX_glcn_e',
      'D-galactose': 'EX_gal_e'},
  5: {'D-ribose': 'EX_rib__D_e',
      'L-arabinose': 'EX_arab__L_e',
      'L-rhamnose': 'EX_rmn_e',
      'D-xylose': 'EX_xyl__D_e',
      'L-glutamate': 'EX_glu__L_e'},
  4: {'D-malate': 'EX_mal__D_e',
      'L-malate': 'EX_mal__L_e',
      'succinate': 'EX_succ_e',
      'acetoacetate': 'EX_acac_e'},
  3: {'D-alanine': 'EX_ala__D_e',
      'L-alanine': 'EX_ala__L_e',
      'glycerol': 'EX_glyc_e',
      'pyruvate': 'EX_pyr_e', 
      'L-lactate': 'EX_lac__D_e',
      'D-lactate': 'EX_lac__L_e'},
  2: {'acetate': 'EX_ac_e',
      'acetaladehyde': 'EX_acald_e'}}

output_d = {'carbon_source': [],
            'anaerobic': [],
            'total_nadp_flux': [],
            'total_other_redox_flux': [],
            'flux_fraction': []}

keys = sorted(csources.keys())
for anaerobic in (True, False):
    for carbons in keys:
        for csource_name, rxn_id in csources[carbons].iteritems():
            soln = solve_w_csource(model, rxn_id, anaerobic=anaerobic)
            total_nadp_flux, total_non_nad_flux, fraction_of_total = calc_redox_fluxes(soln)
            output_d['carbon_source'].append(csource_name)
            output_d['anaerobic'].append(anaerobic)
            output_d['total_nadp_flux'].append(total_nadp_flux)
            output_d['total_other_redox_flux'].append(total_non_nad_flux)
            output_d['flux_fraction'].append(fraction_of_total)

df = pd.DataFrame(output_d)
df.to_csv('FBA_redox_flux_ratios.csv')

model could not be solved
csource = EX_ac_e, anaerobic = True
model could not be solved
csource = EX_lac__D_e, anaerobic = True
model could not be solved
csource = EX_lac__L_e, anaerobic = True
model could not be solved
csource = EX_succ_e, anaerobic = True
